In [1]:
%pip install xgboost
%pip install scikit-learn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.feature_selection import chi2
from scipy.stats import spearmanr
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import mutual_info_classif


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# データ(csvファイル)の読み込み
df = pd.read_csv('/Users/yasudayuuya/AI_ethics/file/Vaccine_Hesitancy_for_COVID-19__County_and_local_estimates_20250227.csv')
df.head()

,FIPS Code,County Name,State,Estimated hesitant,Estimated hesitant or unsure,Estimated strongly hesitant,Social Vulnerability Index (SVI),SVI Category,CVAC level of concern for vaccination rollout,CVAC Level Of Concern,...,Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Geographical Point,State Code,County Boundary,State Boundary
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.2400,0.1383,0.89,Very High Vulnerability,0.64,High Concern,...,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-85.841259 33.104456, -85.8409...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.2350,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,...,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-86.303069 33.46316, -86.30306...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,...,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.52534299999999 32.132773, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
3,1129,"Washington County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.73,High Vulnerability,0.82,Very High Concern,...,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.45317899999999 31.505388, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
4,1119,"Sumter County, Alabama",ALABAMA,0.1808,0.2526,0.1390,0.96,Very High Vulnerability,0.96,Very High Concern,...,0.0053,0.0007,0.0214,0.7192,0.0000,0.2481,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.369055 32.742584, -88.3685...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."


In [3]:
# データの前処理
# 欠損値の確認
df.isnull().sum()

FIPS Code                                                             0
County Name                                                           0
State                                                                 0
Estimated hesitant                                                    0
Estimated hesitant or unsure                                          0
Estimated strongly hesitant                                           0
Social Vulnerability Index (SVI)                                      1
SVI Category                                                          1
CVAC level of concern for vaccination rollout                         0
CVAC Level Of Concern                                                 0
Percent adults fully vaccinated against COVID-19 (as of 6/10/21)    278
Percent Hispanic                                                      0
Percent non-Hispanic American Indian/Alaska Native                    0
Percent non-Hispanic Asian                                      

In [4]:
# 欠損値のあった列のみ出力
df.isnull().sum()[df.isnull().sum()>0]

Social Vulnerability Index (SVI)                                      1
SVI Category                                                          1
Percent adults fully vaccinated against COVID-19 (as of 6/10/21)    278
County Boundary                                                       1
State Boundary                                                        1
dtype: int64

In [5]:
# 欠損値のある行を削除
df = df.dropna()
df.isnull().sum()
df

,FIPS Code,County Name,State,Estimated hesitant,Estimated hesitant or unsure,Estimated strongly hesitant,Social Vulnerability Index (SVI),SVI Category,CVAC level of concern for vaccination rollout,CVAC Level Of Concern,...,Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Geographical Point,State Code,County Boundary,State Boundary
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.2400,0.1383,0.89,Very High Vulnerability,0.64,High Concern,...,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-85.841259 33.104456, -85.8409...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.2350,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,...,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-86.303069 33.46316, -86.30306...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,...,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.52534299999999 32.132773, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
3,1129,"Washington County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.73,High Vulnerability,0.82,Very High Concern,...,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.45317899999999 31.505388, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
4,1119,"Sumter County, Alabama",ALABAMA,0.1808,0.2526,0.1390,0.96,Very High Vulnerability,0.96,Very High Concern,...,0.0053,0.0007,0.0214,0.7192,0.0000,0.2481,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.369055 32.742584, -88.3685...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,55079,"Milwaukee County, Wisconsin",WISCONSIN,0.1505,0.2144,0.0880,0.81,Very High Vulnerability,0.30,Low Concern,...,0.1500,0.0047,0.0428,0.2606,0.0002,0.5124,POINT (-89.732969 44.639954),WI,"MULTIPOLYGON (((-87.994171 43.19255, -87.99331...","MULTIPOLYGON (((-92.887067 45.644148, -92.8825..."
3138,55121,"Trempealeau County, Wisconsin",WISCONSIN,0.1611,0.2126,0.0959,0.28,Low Vulnerability,0.31,Low Concern,...,0.0840,0.0034,0.0043,0.0051,0.0000,0.8953,POINT (-89.732969 44.639954),WI,"MULTIPOLYGON (((-91.529034 44.422101, -91.5292...","MULTIPOLYGON (((-92.887067 45.644148, -92.8825..."
3139,56001,"Albany County, Wyoming",WYOMING,0.1949,0.2673,0.1405,0.25,Low Vulnerability,0.63,High Concern,...,0.0953,0.0091,0.0327,0.0150,0.0003,0.8248,POINT (-107.55145 42.999627),WY,MULTIPOLYGON (((-106.071399 41.530719999999995...,"MULTIPOLYGON (((-111.044893 43.315719, -111.04..."
3140,55067,"Langlade County, Wisconsin",WISCONSIN,0.1518,0.2007,0.0906,0.35,Low Vulnerability,0.19,Very Low Concern,...,0.0197,0.0069,0.0022,0.0125,0.0002,0.9383,POINT (-89.732969 44.639954),WI,"MULTIPOLYGON (((-89.42472 45.293175, -89.42473...","MULTIPOLYGON (((-92.887067 45.644148, -92.8825..."


In [6]:
# ターゲット変数設定（ここではTotal PrevalenceのRateを予測ターゲットとします）
target = 'Estimated strongly hesitant'
df[target] = df[target].astype(float)  # 文字列を浮動小数点数に変換
# 特徴量設定
features = df.drop(columns=['FIPS Code', 'State', 'Estimated hesitant', 'Estimated hesitant or unsure', 'SVI Category', 'CVAC Level Of Concern', 'Percent adults fully vaccinated against COVID-19 (as of 6/10/21)',  'Geographical Point', 'State Code', 'County Boundary', 'State Boundary', target])


In [7]:
# 説明変数のカラム名を表示
#　表形式で表示
features.columns

Index(['County Name', 'Social Vulnerability Index (SVI)',
       'CVAC level of concern for vaccination rollout', 'Percent Hispanic',
       'Percent non-Hispanic American Indian/Alaska Native',
       'Percent non-Hispanic Asian', 'Percent non-Hispanic Black',
       'Percent non-Hispanic Native Hawaiian/Pacific Islander',
       'Percent non-Hispanic White'],
      dtype='object')

In [8]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# 特徴量のカラム名から特殊文字を削除
# 今回削除される文字は、[ ] % , です
features.columns = features.columns.str.replace('[\[\]%, ]', '_', regex=True)


# カテゴリ変数をラベルエンコーディング
# これによって、カテゴリ変数を数値に変換することができます
# ここにあるcounty_nameは数値に変換すると
label_encoder = LabelEncoder()
for col in features.select_dtypes(include=['object']).columns:
    features[col] = label_encoder.fit_transform(features[col])

# County_Nameをエンコーディング
features['County_Name'] = label_encoder.fit_transform(features['County_Name'])

# ターゲット変数を2値化
# 今回のターゲット変数は連続値ですが、2値化して分類問題として扱います
median_value = df[target].median()
binary_target = (df[target] > median_value).astype(int)

# データセットの分割
X_train, X_test, y_train, y_test = train_test_split(features, df[target], test_size=0.2, random_state=42)


In [9]:
X_train['Social_Vulnerability_Index_(SVI)'].head(10) # これによって、Social Vulnerability Index (SVI)の数値と名前の対応を確認できます

2129    0.40
753     0.31
2342    0.05
2165    0.12
773     0.56
1145    0.18
1629    0.61
2590    0.11
2409    0.82
562     0.21
Name: Social_Vulnerability_Index_(SVI), dtype: float64

In [10]:
X_train['Percent_Hispanic'].head(10) # この数値は、ヒスパニックの割合を示しています

2129    0.0274
753     0.0498
2342    0.0276
2165    0.0205
773     0.0182
1145    0.0192
1629    0.0119
2590    0.0245
2409    0.4591
562     0.0428
Name: Percent_Hispanic, dtype: float64

In [11]:
# XGBoostの適用と特徴量重要度の計算
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_model.fit(X_train, y_train)

# 特徴量重要度の抽出
xgb_importances = xgb_model.feature_importances_
xgb_indices = xgb_importances.argsort()[::-1]
xgb_top_features = [(features.columns[i], xgb_importances[i]) for i in xgb_indices[:5]]

# 結果の表示
print("XGBoost Top 5 Features:")
for feature, importance in xgb_top_features:
    print(f"{feature}: {importance:.6f}")

XGBoost Top 5 Features:
CVAC_level_of_concern_for_vaccination_rollout: 0.204054
Percent_Hispanic: 0.150387
Percent_non-Hispanic_Asian: 0.131317
Percent_non-Hispanic_Black: 0.111441
Percent_non-Hispanic_American_Indian/Alaska_Native: 0.109138


In [12]:
# Chi-squared Test
chi2_scores, chi2_pvalues = chi2(X_train, binary_target.loc[X_train.index])
chi2_indices = chi2_scores.argsort()[::-1]
chi2_top_features = [(features.columns[i], chi2_scores[i], chi2_pvalues[i]) for i in chi2_indices[:5]]

# 結果の表示
print("Chi-squared Top 5 Features:")
for i, (feature, score, p) in enumerate(chi2_top_features, start=1):
    print(f"{feature}: {score:.6f} (p-value: {p:.6e})")

Chi-squared Top 5 Features:
County_Name: 150.220180 (p-value: 1.551794e-34)
CVAC_level_of_concern_for_vaccination_rollout: 76.293800 (p-value: 2.444502e-18)
Social_Vulnerability_Index_(SVI): 46.989773 (p-value: 7.135809e-12)
Percent_non-Hispanic_Black: 19.787701 (p-value: 8.653773e-06)
Percent_non-Hispanic_American_Indian/Alaska_Native: 13.614650 (p-value: 2.244273e-04)


In [13]:
# Spearman's Correlation
spearman_scores = []
for feature in features.columns:
    coef, p = spearmanr(features[feature], df[target])
    spearman_scores.append((feature, coef, p))

spearman_scores.sort(key=lambda x: abs(x[1]), reverse=True)
spearman_top_features = spearman_scores[:5]

# 結果の表示
print("Spearman's Correlation Top 5 Features:")
for feature, coef, p in spearman_top_features:
    print(f"{feature}: {coef:.6f} (p-value: {p:.6e})")

Spearman's Correlation Top 5 Features:
CVAC_level_of_concern_for_vaccination_rollout: 0.447058 (p-value: 1.121011e-140)
Social_Vulnerability_Index_(SVI): 0.360366 (p-value: 1.681309e-88)
Percent_non-Hispanic_Asian: -0.301865 (p-value: 2.265765e-61)
Percent_Hispanic: -0.176135 (p-value: 2.244516e-21)
Percent_non-Hispanic_American_Indian/Alaska_Native: 0.111354 (p-value: 2.326742e-09)


In [14]:
# DataFrame表示オプションの調整
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# 各手法の結果を縦一列にまとめる
results = []

# XGBoostの結果を追加
for i, (feature, importance) in enumerate(xgb_top_features, start=1):
    results.append({'Method': 'XGBoost', 'Rank': i, 'Feature': feature, 'Importance': importance})

# Chi-squaredの結果を追加
for i, (feature, score, p) in enumerate(chi2_top_features, start=1):
    results.append({'Method': 'Chi-squared', 'Rank': i, 'Feature': feature, 'Importance': score, 'P-value': p})

# Spearmanの結果を追加
for i, (feature, coef, p) in enumerate(spearman_top_features, start=1):
    results.append({'Method': 'Spearman', 'Rank': i, 'Feature': feature, 'Importance': coef, 'P-value': p})

# 結果をDataFrameに変換
comparison_df = pd.DataFrame(results)

# 表形式でインデックスなしで表示
print(comparison_df.to_string(index=False))

     Method  Rank                                            Feature  Importance       P-value
    XGBoost     1      CVAC_level_of_concern_for_vaccination_rollout    0.204054           NaN
    XGBoost     2                                   Percent_Hispanic    0.150387           NaN
    XGBoost     3                         Percent_non-Hispanic_Asian    0.131317           NaN
    XGBoost     4                         Percent_non-Hispanic_Black    0.111441           NaN
    XGBoost     5 Percent_non-Hispanic_American_Indian/Alaska_Native    0.109138           NaN
Chi-squared     1                                        County_Name  150.220180  1.551794e-34
Chi-squared     2      CVAC_level_of_concern_for_vaccination_rollout   76.293800  2.444502e-18
Chi-squared     3                   Social_Vulnerability_Index_(SVI)   46.989773  7.135809e-12
Chi-squared     4                         Percent_non-Hispanic_Black   19.787701  8.653773e-06
Chi-squared     5 Percent_non-Hispanic_American_In

In [15]:
# ランダムフォレストの特徴量重要度の計算
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(features, binary_target)

# 特徴量重要度の抽出
importances = model.feature_importances_
feature_importances = pd.DataFrame({'Feature': features.columns, 'Importance': importances})
feature_importances.sort_values(by='Importance', ascending=False, inplace=True)

# 結果の表示
print("Random Forest Top 5 Important Features:")
print(feature_importances.head(5))

Random Forest Top 5 Important Features:
                                              Feature  Importance
2       CVAC_level_of_concern_for_vaccination_rollout    0.220989
1                    Social_Vulnerability_Index_(SVI)    0.126515
3                                    Percent_Hispanic    0.121242
5                          Percent_non-Hispanic_Asian    0.109359
4  Percent_non-Hispanic_American_Indian/Alaska_Native    0.100802


In [16]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 複数のアルファ値を試してみる
alphas = [0.1, 0.01, 0.001, 0.0001]
results = []

# 定義済みの features と df[target] を使用
X = features
y = df[target]

for alpha in alphas:
    lasso = Lasso(alpha=alpha, random_state=42)
    pipeline = Pipeline([('scaler', StandardScaler()), ('lasso', lasso)])
    pipeline.fit(X, y)
    
    # 非ゼロの係数を持つ特徴量を取得
    lasso_coef = pipeline.named_steps['lasso'].coef_
    lasso_importances = pd.Series(lasso_coef, index=features.columns)
    non_zero_importances = lasso_importances[lasso_importances != 0].sort_values(ascending=False)
    
    results.append((alpha, non_zero_importances))

# 結果を表示
for alpha, importances in results:
    print(f"アルファ: {alpha}")
    print("LASSO特徴量の重要度:")
    print(importances.head(5))
    print('-' * 40)

アルファ: 0.1
LASSO特徴量の重要度:
Series([], dtype: float64)
----------------------------------------
アルファ: 0.01
LASSO特徴量の重要度:
CVAC_level_of_concern_for_vaccination_rollout    0.004081
dtype: float64
----------------------------------------
アルファ: 0.001
LASSO特徴量の重要度:
CVAC_level_of_concern_for_vaccination_rollout            0.009827
Social_Vulnerability_Index_(SVI)                         0.004365
Percent_non-Hispanic_American_Indian/Alaska_Native       0.002798
Percent_non-Hispanic_Native_Hawaiian/Pacific_Islander    0.001115
Percent_non-Hispanic_Asian                              -0.004641
dtype: float64
----------------------------------------
アルファ: 0.0001
LASSO特徴量の重要度:
CVAC_level_of_concern_for_vaccination_rollout            0.010248
Social_Vulnerability_Index_(SVI)                         0.006148
Percent_non-Hispanic_American_Indian/Alaska_Native       0.002990
Percent_non-Hispanic_Native_Hawaiian/Pacific_Islander    0.002138
County_Name                                             -0.000552


In [17]:
from sklearn.model_selection import GridSearchCV

# グリッドサーチの設定
param_grid = {'lasso__alpha': [0.1, 0.01, 0.001, 0.0001]}
lasso = Lasso(random_state=42)
pipeline = Pipeline([('scaler', StandardScaler()), ('lasso', lasso)])
grid_search = GridSearchCV(pipeline, param_grid, cv=5)

# グリッドサーチの実行
grid_search.fit(X, y)

# 最適なパラメータを取得
best_alpha = grid_search.best_params_['lasso__alpha']
print(f"最適なアルファ値: {best_alpha}")

# 最適なアルファ値を用いたLASSOのトレーニング
best_lasso = Lasso(alpha=best_alpha, random_state=42)
pipeline = Pipeline([('scaler', StandardScaler()), ('lasso', best_lasso)])
pipeline.fit(X, y)

# 非ゼロの係数を持つ特徴量を取得
lasso_coef = pipeline.named_steps['lasso'].coef_
lasso_importances = pd.Series(lasso_coef, index=features.columns)
lasso_importances = lasso_importances[lasso_importances != 0].sort_values(ascending=False)

# 結果を表示
print("最適なアルファ値でのLASSO特徴量の重要度:")
print(lasso_importances.head(5))

最適なアルファ値: 0.001
最適なアルファ値でのLASSO特徴量の重要度:
CVAC_level_of_concern_for_vaccination_rollout            0.009827
Social_Vulnerability_Index_(SVI)                         0.004365
Percent_non-Hispanic_American_Indian/Alaska_Native       0.002798
Percent_non-Hispanic_Native_Hawaiian/Pacific_Islander    0.001115
Percent_non-Hispanic_Asian                              -0.004641
dtype: float64


In [18]:
from sklearn.svm import SVC

# SVMモデルの作成
svm = SVC(kernel="linear", random_state=42)
svm.fit(X, binary_target)

# モデルから係数を取得
coefficients = svm.coef_[0]

# 特徴量とその係数をデータフレームにまとめる
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': coefficients
})

# 係数の絶対値でランキング
feature_importance['Abs_Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values(by='Abs_Coefficient', ascending=False)

# 上位5つの特徴量を選択
top_5_features = feature_importance.head(5).copy()

# ランキング列を追加
top_5_features['Ranking'] = range(1, 6)

# 不要な列を削除
top_5_features = top_5_features.drop(columns=['Abs_Coefficient'])

print("SVM トップ5特徴量とランキング、係数:")
for index, row in top_5_features.iterrows():
    print(f"{row['Feature']}: ランキング {row['Ranking']}, 係数 {row['Coefficient']:.6f}")

SVM トップ5特徴量とランキング、係数:
Percent_Hispanic: ランキング 1, 係数 -10.688914
CVAC_level_of_concern_for_vaccination_rollout: ランキング 2, 係数 10.555800
Percent_non-Hispanic_White: ランキング 3, 係数 6.970293
Percent_non-Hispanic_American_Indian/Alaska_Native: ランキング 4, 係数 6.344827
Social_Vulnerability_Index_(SVI): ランキング 5, 係数 5.643722


In [19]:
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import mutual_info_classif

# miは相互情報量を表す
# これによって、特徴量とターゲット変数の間の依存関係を計算できます
# この関数は、特徴量が連続値でもカテゴリ値でも機能します
mi = mutual_info_classif(X, binary_target)
mi_series = pd.Series(mi, index=features.columns)
mi_series.sort_values(ascending=False, inplace=True)

print("Naive Bayes (Mutual Information) Top Features:")
print(mi_series.head(5))

Naive Bayes (Mutual Information) Top Features:
CVAC_level_of_concern_for_vaccination_rollout    0.116944
Social_Vulnerability_Index_(SVI)                 0.075737
Percent_non-Hispanic_Asian                       0.038228
Percent_non-Hispanic_Black                       0.030976
Percent_Hispanic                                 0.030913
dtype: float64


In [20]:
from scipy.stats import kendalltau

# Kendall's Tauとp値を計算し、DataFrameを作成
kendall_results = []
for column in X.columns:
    tau, p_value = kendalltau(X[column], y)
    kendall_results.append((column, tau, p_value))

# DataFrameに変換
kendall_df = pd.DataFrame(kendall_results, columns=['Feature', 'Kendall_Tau', 'P_value'])

# Kendall's Tauの絶対値でソート（1に近いほど強い相関）
kendall_df['Abs_Tau'] = kendall_df['Kendall_Tau'].abs()
sorted_kendall_df = kendall_df.sort_values(by='Abs_Tau', ascending=False)

# 上位5つの特徴量をフィルタリング
top_kendall_features = sorted_kendall_df.head(5)

print("Kendall's Tau 上位5特徴量とP値:")
for index, row in top_kendall_features.iterrows():
    print(f"{row['Feature']}: Kendall's Tau {row['Kendall_Tau']:.6f}, P-value {row['P_value']:.6e}")

Kendall's Tau 上位5特徴量とP値:
CVAC_level_of_concern_for_vaccination_rollout: Kendall's Tau 0.303825, P-value 1.490570e-129
Social_Vulnerability_Index_(SVI): Kendall's Tau 0.248004, P-value 5.474969e-87
Percent_non-Hispanic_Asian: Kendall's Tau -0.208356, P-value 5.563521e-62
Percent_Hispanic: Kendall's Tau -0.117180, P-value 6.508811e-21
Percent_non-Hispanic_American_Indian/Alaska_Native: Kendall's Tau 0.075049, P-value 2.374459e-09


In [22]:
%pip install pandas statsmodels

  Using cached statsmodels-0.14.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
Using cached statsmodels-0.14.4-cp310-cp310-macosx_11_0_arm64.whl (9.9 MB)
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.


In [23]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# VIFを計算する関数
def calculate_vif(df):
    X = sm.add_constant(df)
    vif_data = pd.DataFrame()
    vif_data['feature'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif_data = vif_data[vif_data['feature'] != 'const']
    return vif_data.sort_values(by='VIF', ascending=False)

# 特徴量の重要度を計算する関数
def get_feature_importance(X, y):
    model = RandomForestRegressor(random_state=42)
    model.fit(X, y)
    importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    })
    return importance.sort_values(by='importance', ascending=True)

# 特徴量の重要度の低い順に削除し、各ステップのVIFを確認
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)
features_to_remove = get_feature_importance(X_train, y_train)['feature'].tolist()

removed_features = []
for i in range(len(features_to_remove)):
    temp_features = X_train.drop(columns=removed_features, errors='ignore')
    if len(temp_features.columns) == 0:
        break
        
    vif_data = calculate_vif(temp_features)
    
    # VIFが全て5以下か確認
    if (vif_data['VIF'] <= 5).all():
        print(f"All VIFs are 5 or below. Stopping the process.")
        break
    
    # 重要度の低い特徴量を一つ削除
    removed_features.append(features_to_remove[i])
    print(f"Removed {features_to_remove[i]}\n")
    print(vif_data)
    print("\n")

# 最終的な特徴量
print("Final features:")
print(temp_features.columns)

# 結果のVIF
final_vif = calculate_vif(temp_features)
print(final_vif)

Removed Percent_non-Hispanic_Native_Hawaiian/Pacific_Islander

                                                 feature         VIF
9                             Percent_non-Hispanic_White  166.314629
7                             Percent_non-Hispanic_Black  101.872028
4                                       Percent_Hispanic   43.713922
5     Percent_non-Hispanic_American_Indian/Alaska_Native   33.191911
6                             Percent_non-Hispanic_Asian    5.324082
2                       Social_Vulnerability_Index_(SVI)    2.744435
3          CVAC_level_of_concern_for_vaccination_rollout    2.241567
8  Percent_non-Hispanic_Native_Hawaiian/Pacific_Islander    1.198886
1                                            County_Name    1.002960


Removed County_Name

                                              feature         VIF
8                          Percent_non-Hispanic_White  156.819002
7                          Percent_non-Hispanic_Black   95.408462
4                         

In [24]:
# 結果のVIF
final_vif = calculate_vif(temp_features)
print(final_vif)

                                              feature       VIF
1                    Social_Vulnerability_Index_(SVI)  2.733834
6                          Percent_non-Hispanic_White  2.619284
2       CVAC_level_of_concern_for_vaccination_rollout  2.219467
3                                    Percent_Hispanic  1.391988
5                          Percent_non-Hispanic_Asian  1.288168
4  Percent_non-Hispanic_American_Indian/Alaska_Native  1.221718
